<a href="https://colab.research.google.com/github/Naomine14/--1/blob/main/%D0%A7%D0%B5%D0%BA%D0%B0%D0%BD%D0%B8%D0%BD%D0%B0_%D0%98%D0%A232_%D0%9B3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Итак, перед тем, как браться за статистику, нужно:

**1. Прочесть исходный файл и превратить его в структуру данных**

К заданию прилагается файл в формате csv, где все значения разделены запятыми. Это наши исходные данные. Чтобы применить к ним все возможности языка Python и библиотеки Pandas, надо импортировать эту библиотеку и сохранить её в переменной. По сокращённому названию панельных данных (panel data), с которых начиналась Pandas, эту переменную принято называть pd:

**import pandas as pd**

Для чтения csv-файла в библиотеке Pandas есть готовая функция — метод **read_csv()**. Как и все методы, он вызывается записью через точку после имени своего объекта. В скобках указывается аргумент (параметр) метода. У read_csv() это имя файла с данными. Прочтение превращает файл в структуру данных DataFrame. Имя переменной, в которой эта структура данных сохраняется, чаще всего df либо отражает тематику данных:

**df = pd.read_csv('music_log.csv')**

**2. Посмотреть на данные**

Вывести на экран таблицу и оценить данные:

print(df)
Как правило, таблица очень велика. Практичнее запросить определённое количество первых строк, методом head().

**3. Оценить качество предподготовки**

Нужно убедиться в том, что данные прошли предподготовку. По крайней мере, не должно быть пропусков и повторов. Пропущенные и неопределённые значения выявляет метод **isna()**, а суммарное количество таких значений — метод **sum()**.

**print(df.isna().sum())**


Повторяющиеся строки — дубликаты — выявляются методом **duplicated()** и подсчитываются тем же sum():

**print(df.duplicated().sum())**
Если возвращаются нули, всё хорошо — данные пригодны для исследования.


## Задача

1. Загрузите таблицу **exoplanet_catalog**, содержащую данные о планетах за пределами Солнечной системы, или экзопланетах. Сформируйте из данной таблицы новый датасет под назанием **exoplanet**, коорая будет содердать стоблцы name, mass, radius, discovered. Проверьте, чтобы название всех столбцов было корректным, при необходмости измените названия.

2. Удалите из получившейся таблица строки с нулевыми значениями.

3. Убедитесь, что таблица не содержит ни пропущенных значений, ни дубликатов.

In [ ]:
import pandas as pd

# Задача 1
exoplanet_catalog = pd.read_csv('exoplanet_catalog.csv')

exoplanet = exoplanet_catalog[['name', 'mass', 'radius', 'discovered']].copy()

# Проверка и коррекция названий столбцов
exoplanet.columns = exoplanet.columns.str.strip().str.lower()

# Задача 2
exoplanet = exoplanet.dropna()

# Проверка наличия пропущенных значений
print('Проверка наличия пропущенных значений\n', exoplanet.isnull().sum())

# Удаление дубликатов
exoplanet = exoplanet.drop_duplicates()

# Задача 3
print('Проверка наличия пропущенных значений и дубликатов',exoplanet.duplicated().sum())


Проверка наличия пропущенных значений
 name          0
mass          0
radius        0
discovered    0
dtype: int64
Проверка наличия пропущенных значений и дубликатов 0


# Группировка данных

Слово «анализ» означает разбор, рассмотрение с разных сторон. Анализ данных начинают с разделения их на группы по какому-нибудь признаку. Эта операция называется группировка данных. Она помогает изучить материал более подробно, чтобы затем перейти к поиску взаимосвязей между отдельными группами.


Группировка оправданна, если данные чётко делятся по значимому признаку, а полученные группы близки к теме задачи. Например, когда есть данные обо всех покупках в супермаркете, можно смело заниматься группировкой. Так можно установить время наплыва покупателей и решить проблему пиковых нагрузок. Или посчитать средний чек — обычно для магазинов это ключевая метрика.


Стадии группировки хорошо укладываются в словесную формулу **split-apply-combine**:

•	разделить, **split** — разбиение на группы по определённому критерию;

•	применить, **apply** — применение какого-либо метода к каждой группе в отдельности, например, подсчёт численности группы методом count() или суммирование вызовом sum();

•	объединить, **combine** — сведение результатов в новую структуру данных, в зависимости от условий разделения и выполнения метода это бывает DataFrame и Series.

В библиотеке Pandas есть отличные инструменты группировки. Рассмотрим обращение с ними на примере анализа данных о планетах за пределами Солнечной системы, или экзопланетах. Орбитальные обсерватории засекли уже тысячи таких небесных тел. Их выявляют на снимках космических телескопов наши коллеги, аналитики данных. Поищем среди экзопланет похожие на Землю. Возможно, это наши будущие колонии, или там уже обитают разумные существа, с которыми однажды предстоит установить контакт.

DataFrame с данными по нескольким тысячам экзопланет сохранён в переменной **exoplanet**. Посмотрим на первые 30 строк таблицы:


In [ ]:
print(exoplanet_catalog.head(30))

                                                 name planet_status  mass  \
0                                            11 Com b     Confirmed   NaN   
1   11 Oph b,Confirmed,21.0,3.0,3.0,,,,,,,730000.0...           NaN   NaN   
2                                            11 UMi b     Confirmed   NaN   
3                                            14 And b     Confirmed   NaN   
4                                            14 Her b     Confirmed   NaN   
5                                            14 Her c     Confirmed   5.8   
6                                          16 Cyg B b     Confirmed   NaN   
7                                            17 Sco b     Confirmed   NaN   
8                                            18 Del b     Confirmed   NaN   
9   1I/'Oumuamua,Confirmed,,,,,,,2e-06,,,,,,,,,1.1...           NaN   NaN   
10  1RXS 1609 b,Confirmed,14.0,3.0,2.0,,,,1.7,,,,,...           NaN   NaN   
11                            1RXS J235133.3+312720 b     Confirmed  32.0   

**Документация**

Столбцы:

•	name: название экзопланеты;

•	mass: масса в массах планеты Юпитер;

•	radius: радиус, пересчитанный в радиусах Земли;

•	discovered: год открытия экзопланеты.

*Источник: каталог экзопланет на портале exoplanet.eu*

На картинке изображен принцип **split-apply-combine** для таблицы с экзопланетами. Посмотрим, как вообще идут дела с поиском экзопланет. Сначала данные делят по группам, где каждая группа — это год. Потом метод **count()** подсчитывает численность каждой группы. В итоге получаем новую структуру данных с группами, где каждая содержит год и число открытых за этот год экзопланет.


В Рandas для группировки данных есть метод **groupby()**. Он принимает как аргумент название столбца, по которому нужно группировать. В случае с делением экзопланет по годам открытия:


**print(exoplanet.groupby('discovered'))**

**<pandas.core.groupby.DataFrameGroupBy object at 0x7fc1e1ca3400>**


Применение метода **groupby()** к объекту типа DataFrame приводит к созданию объекта особого типа — **DataFrameGroupBy**. Это сгруппированные данные. Если применить к ним какой-нибудь метод Pandas, они станут новой структурой данных типа **DataFrame** или **Series**.
Подсчитаем сгруппированные по годам экзопланеты методом **count()**:


In [ ]:
print(exoplanet.groupby('discovered').count())


            name  mass  radius
discovered                    
1996.0         1     1       1
2000.0         2     2       2
2001.0         1     1       1
2002.0         1     1       1
2004.0         5     5       5
2005.0         3     3       3
2006.0         6     6       6
2007.0        18    18      18
2008.0        18    18      18
2009.0         8     8       8
2010.0        41    41      41
2011.0        45    45      45
2012.0        24    24      24
2013.0        27    27      27
2014.0        20    20      20
2015.0        22    22      22
2016.0        45    45      45
2017.0        31    31      31
2018.0        60    60      60
2019.0        51    51      51
2020.0        79    79      79
2021.0        42    42      42


Если нужно сравнить наблюдения по одному показателю, метод применяют к **DataFrameGroupBy** с указанием на один столбец. Нас в первую очередь интересует радиус экзопланет: мы ищем другую Землю. Давайте получим таблицу с единственным столбцом 'radius':

**exo_number = exoplanet.groupby('discovered')['radius'].count()**

**print(exo_number)**

In [ ]:
exo_number = exoplanet.groupby('discovered')['radius'].count()
print(exo_number)

discovered
1996.0     1
2000.0     2
2001.0     1
2002.0     1
2004.0     5
2005.0     3
2006.0     6
2007.0    18
2008.0    18
2009.0     8
2010.0    41
2011.0    45
2012.0    24
2013.0    27
2014.0    20
2015.0    22
2016.0    45
2017.0    31
2018.0    60
2019.0    51
2020.0    79
2021.0    42
Name: radius, dtype: int64


Получили Series, где по годам открытия расписано количество экзопланет, для которых удалось установить радиус.

Посмотрим, как меняется средний радиус открытых экзопланет год от года. Для этого надо сложить радиусы планет, открытых за определённый год, и поделить на их количество (которое мы уже нашли).

Сумма радиусов считается методом **sum()**:

**exo_radius_sum = exoplanet.groupby('discovered')['radius'].sum()**


**print(exo_radius_sum)**



In [ ]:
exo_radius_sum = exoplanet.groupby('discovered')['radius'].sum()

print(exo_radius_sum)


discovered
1996.0     1.060000
2000.0     2.007000
2001.0     0.921000
2002.0     1.200000
2004.0     6.236000
2005.0     3.651000
2006.0    15.863000
2007.0    23.361600
2008.0    24.179000
2009.0     6.091794
2010.0    46.309490
2011.0    48.860674
2012.0    29.561700
2013.0    35.518800
2014.0    22.925000
2015.0    26.512240
2016.0    55.220000
2017.0    29.684920
2018.0    65.627080
2019.0    48.620800
2020.0    60.420350
2021.0    31.915760
Name: radius, dtype: float64


Очень кстати, что объекты Series можно делить друг на друга. Это позволит нам разделить перечень сумм радиусов на перечень количеств экзопланет без перебора в цикле:


**exo_radius_mean = exo_radius_sum/exo_number**

**print(exo_radius_mean)**


In [ ]:
exo_radius_mean = exo_radius_sum/exo_number

print(exo_radius_mean)


discovered
1996.0    1.060000
2000.0    1.003500
2001.0    0.921000
2002.0    1.200000
2004.0    1.247200
2005.0    1.217000
2006.0    2.643833
2007.0    1.297867
2008.0    1.343278
2009.0    0.761474
2010.0    1.129500
2011.0    1.085793
2012.0    1.231737
2013.0    1.315511
2014.0    1.146250
2015.0    1.205102
2016.0    1.227111
2017.0    0.957578
2018.0    1.093785
2019.0    0.953349
2020.0    0.764815
2021.0    0.759899
Name: radius, dtype: float64


Точность наших приборов растёт, и новые экзопланеты по размерам всё ближе к Земле.

А теперь вернемся к анализу датафрейма с музыкльными предпочтениями слушателей Яндекс.Музыки. Идею объединения сервисов Музыка и Радио тестировали на небольшой группе пользователей. Результаты сведены в csv-файл, который вам предстоит изучить. Итог анализа таких данных — это метрики: величины, значения которых отражают пользовательские впечатления. Одна из важнейших — **happiness**. Здесь это среднее время, которое пользователь слушает музыку в течение выбранного периода времени (в нашей задаче — за сутки). Чем дольше пользователь слушает музыку, тем он довольнее. Ваша задача: найти значение happiness и посмотреть, как оно менялось.

Тем же методом **groupby()**, которым мы ищем новую Землю, можно поискать и необыкновенного человека в данных Яндекс.Музыки. Тем более, что без этого не выполнить поставленной менеджером задачи.

Прежде, чем рассчитывать метрику happiness, нужно изучить пользователей, чьё «счастье» мы собираемся оценить. Какие они, эти люди, которые слушают действительно много музыки? Есть ли у них особые предпочтения, или они потребляют всё подряд?


In [ ]:
#обработка данных для дальнейшей работы

import pandas as pd

yandex_music = pd.read_csv('yandex_music.csv')

new_names = {
'  userID': 'user_id',
'Track': 'track_name',
'artist': 'artist_name',
'genre': 'genre_name',
'  City  ': 'city',
'time': 'total_play_seconds',
'Day': 'day'
}

yandex_music.rename(columns=new_names, inplace=True)

yandex_music.fillna({'track_name':'unknown'}, inplace=True)

yandex_music.fillna({'artist_name':'unknown'}, inplace=True)

yandex_music = yandex_music.dropna(subset=['genre_name'])
yandex_music['genre_name'] = yandex_music['genre_name'].replace('электроника', 'electronic')


## Задача

1. Меломаны у нас есть. Сейчас узнаем идентификатор **user_id** одного из них. Для этого сгруппируем данные по каждому пользователю, чтобы собрать жанры прослушанных им композиций.

Сгруппируйте DataFrame по столбцу user_id, сохраните полученный результат в переменной genre_grouping.

Посчитайте количество жанров, которые выбрали пользователи, методом count(), указав, что выбираем один столбец genre_name.

Сохраните результат в переменной genre_counting и выведите первые 30 строк этой таблицы.

2. Быть может, те, кто за день слушает больше 50 песен, имеют более широкие предпочтения. Чтобы найти такого, изготовим универсальный инструмент.

Напишите функцию user_genres, которая принимает некую группировку как свой аргумент group. Функция должна перебирать группы, входящие в эту группировку.

В каждой группе два элемента — имя группы с индексом 0 и список значений с индексом 1.

Обнаружив такую группу, в которой список (элемент с индексом 1) содержит более 50 значений, функция возвращает имя группы (значение элемента с индексом 0).

3. Вызовите функцию **user_genres**, как аргумент передайте ей **genre_grouping**. Результат – **user_id** неведомого нам любителя музыки – сохраните в переменной **search_id** и выведите значение на экран.


In [ ]:
# Задача 1
genre_grouping = yandex_music.groupby('user_id')

# Посчитаем количество жанров для каждого пользователя
genre_counting = genre_grouping['genre_name'].nunique().reset_index(name='genre_count')
print(genre_counting.head(30))


# Задача 2
def user_genres(group):
    for user_id, songs in group:
        if len(songs) > 50:
            return user_id
    return None


# Задача 3
search_id = user_genres(genre_grouping)
print(search_id)


     user_id  genre_count
0   10004220            3
1   100173ED            1
2   1001CE90            5
3   1002410A            1
4   100261F9            1
5   1004AAE6            1
6   1004C5D8            1
7   1005BECA            1
8   10089377            2
9   100AB021            1
10  100ACFBC            1
11  10101201            2
12  10107FC7            2
13  10112F4C            2
14  1016144E            1
15  1016C39F            2
16  10180175            1
17  101861D1            2
18  10189D10            2
19  101AAA19            1
20  101B046A            2
21  101B53F1            1
22  101BBC61            1
23  101C12A4            2
24  101D714C            1
25  101E910F            1
26  10211433            1
27  10222F55            1
28  1022B77E            1
29  10246DCB            2
7D166C63


# Сортировка данных

Поиск необычного в группе — что среди планет, что среди меломанов — это прежде всего поиск чемпионов: объектов с выдающимися показателями по разным статьям. Как всю таблицу, так и отдельные группы изучают, сортируя строки по какому-либо столбцу.

В Pandas для этой операции есть метод **sort_values()**. У него два аргумента:

• **by = 'имя столбца'** — имя столбца, по которому нужно сортировать;

• **ascending:** по умолчанию True. Для сортировки по убыванию установите значение False.



Среди экзопланет интересны близкие по размерам к Земле. Есть ли такие? Отсортируем список по радиусу в порядке возрастания. Тогда в голове таблицы окажутся самые малые, на которых гравитация не прижмёт нас к полу.

**print(exoplanet.sort_values(by='radius').head(30))**


In [ ]:
print(exoplanet.sort_values(by='radius').head(30))


              name      mass   radius  discovered
4473  TRAPPIST-1 h  0.001041  0.06740      2017.0
4469  TRAPPIST-1 d  0.001300  0.06890      2016.0
4470  TRAPPIST-1 e  0.002000  0.08190      2017.0
4449     TOI-700 b  0.004060  0.09010      2020.0
4471  TRAPPIST-1 f  0.002100  0.09323      2017.0
4468  TRAPPIST-1 c  0.004340  0.09421      2016.0
1699    KOI-2700 b  0.002710  0.09460      2013.0
4467  TRAPPIST-1 b  0.002700  0.09689      2016.0
4108    LHS 1815 b  0.013200  0.09707      2020.0
4472  TRAPPIST-1 g  0.004220  0.10050      2017.0
4404     TOI-178 b  0.004720  0.10278      2020.0
4451     TOI-700 d  0.007110  0.10620      2020.0
218      GJ 1252 b  0.006580  0.10643      2018.0
335      GJ 9827 c  0.006040  0.10700      2017.0
687    HD 158259 b  0.008500  0.10700      2020.0
4107    LHS 1478 b  0.007330  0.11060      2021.0
4425     TOI-270 b  0.006000  0.11125      2019.0
264      GJ 3473 b  0.005850  0.11277      2020.0
4106    LHS 1140 c  0.005690  0.11437      2018.0


Оказывается, некоторые из уже открытых экзопланет по размерам близки не то что к Земле, но уже и к Луне! Получим список экзопланет с радиусом меньше земного. Смотрите, как логический оператор (здесь это <) задействуется в отборе элементов столбца. Дальше нам этот приём не раз понадобится.


**print(exoplanet[exoplanet['radius'] < 1])**

In [ ]:
print(exoplanet[exoplanet['radius'] < 1])

                  name      mass   radius  discovered
22        2M 2140+16 b  20.00000  0.92000      2010.0
41    2MASS J0348-6022  43.00000  0.91000      2021.0
42    2MASS J0407+1546  67.00000  0.97000      2021.0
48    2MASS J1219+3128  49.00000  0.97000      2021.0
85            AU Mic b   0.05300  0.39080      2020.0
...                ...       ...      ...         ...
4662         WASP-64 b   1.20000  0.70000      2011.0
4686         WASP-84 b   0.69400  0.94200      2013.0
4732        Wolf 940 b  26.00000  0.92000      2009.0
4751  ZTF J0038+2030 b  59.30000  0.78300      2021.0
4783          pi Men c   0.01517  0.16719      2018.0

[184 rows x 4 columns]


Но и этот список такой длинный, что изучать его лучше по частям. Экзопланеты, близкие по размерам к Земле, за последнее десятилетие открывали нередко. Можно изучать список открытых за каждый год. Например, для 2014 года (вновь обратите внимание на работу логического оператора, теперь это ==):

**print(exoplanet[exoplanet['discovered'] == 2014])**


In [ ]:
print(exoplanet[exoplanet['discovered'] == 2014])


              name     mass  radius  discovered
342       GU Psc b  11.0000   1.350      2014.0
394     HAT-P-49 b   1.7300   1.413      2014.0
400     HAT-P-54 b   0.7600   0.944      2014.0
427      HATS-15 b   2.1700   1.105      2014.0
454       HATS-4 b   1.3230   1.020      2014.0
465       HATS-5 b   0.2370   0.912      2014.0
477       HATS-6 b   0.3190   0.998      2014.0
1164  HIP 116454 b   0.0372   0.226      2014.0
4521    WASP-104 b   1.2720   1.137      2014.0
4523    WASP-106 b   1.9250   1.085      2014.0
4526    WASP-108 b   1.1670   1.215      2014.0
4527    WASP-109 b   0.9100   1.443      2014.0
4528    WASP-110 b   0.5150   1.238      2014.0
4529    WASP-111 b   1.8500   1.442      2014.0
4530    WASP-112 b   0.8800   1.191      2014.0
4533    WASP-117 b   0.2755   1.021      2014.0
4685     WASP-83 b   0.3000   1.040      2014.0
4688   WASP-87 A b   2.2100   1.385      2014.0
4690     WASP-89 b   5.9000   1.040      2014.0
4695   WASP-94 A b   0.4520   1.720     

А чтобы не тратить время на лишнее, поставим оба условия сразу. Для этого в Pandas есть логический оператор &, подобный оператору and языка Python. Напомним, его смысл на русском языке можно передать словами «и ещё»:

**# экзопланеты меньше Земли __ и ещё __ открытые в 2014 году**

**exo_small_14 = exoplanet[ (exoplanet['radius']<1) & (exoplanet['discovered']==2014)]**

**print(exo_small_14)**



In [ ]:
# экзопланеты меньше Земли и ещё открытые в 2014 году

exo_small_14 = exoplanet[ (exoplanet['radius']<1) & (exoplanet['discovered']==2014)]
print(exo_small_14)

              name    mass  radius  discovered
400     HAT-P-54 b  0.7600   0.944      2014.0
465       HATS-5 b  0.2370   0.912      2014.0
477       HATS-6 b  0.3190   0.998      2014.0
1164  HIP 116454 b  0.0372   0.226      2014.0


Отсортируем результат в порядке убывания радиуса.

**print(exo_small_14.sort_values(by = 'radius', ascending = False))**


In [ ]:
print(exo_small_14.sort_values(by = 'radius', ascending = False))

              name    mass  radius  discovered
477       HATS-6 b  0.3190   0.998      2014.0
400     HAT-P-54 b  0.7600   0.944      2014.0
465       HATS-5 b  0.2370   0.912      2014.0
1164  HIP 116454 b  0.0372   0.226      2014.0


Самая крупная планета, Kepler 106 d – почти как Земля, вращается вокруг звезды Kepler 106 в созвездии Лебедя. Эта звезда очень похожа на наше Солнце. Правда, до неё 1435 световых лет — далековато. Но, возможно, учёные что-нибудь придумают. А мы пока применим эту технологию к нашему бизнесу, в «приземлённой» задаче.
## Задача

1. Космический телескоп Kepler открыл похожую на Землю планету у похожей на Солнце звезды. А вы в данных Яндекс.Музыки обнаружили меломана с уникальными данными. Он за день послушал больше 50 композиций.
Получите таблицу с прослушанными им треками.

Для этого запросите из структуры данных df строки, отвечающие сразу двум условиям:

1) значение в столбце 'user_id' должно быть равно значению переменной search_id;

2) время прослушивания, т.е. значение в столбце 'total_play_seconds', не должно равняться 0.

Сохраните результат в переменной music_user.

2. Теперь узнаем, сколько времени он слушал музыку каждого жанра.

Сгруппируйте данные таблицы music_user по столбцу 'genre_name' и получите сумму значений столбца 'total_play_seconds'. Сохраните результат в переменной sum_music_user и выведите её значение на экран.

3. Кажется, предпочтения нашего меломана начинают проявляться. Но, возможно, длительность композиций от жанра к жанру сильно различается. Важно знать, сколько треков каждого жанра он включил.

Сгруппируйте данные по столбцу genre_name и посчитайте, сколько значений в столбце genre_name. Сохраните результат в переменной count_music_user и выведите её значение на экран.

4. Чтобы предпочтения были видны сразу, нужно крупнейшие значения расположить наверху. Отсортируйте данные в группировке sum_music_user по убыванию. Внимание: когда применяете метод sort_values() к Series с единственным столбцом, аргумент by указывать не нужно, только порядок сортировки.

Сохраните результат в переменной final_sum и выведите её значение на экран.

5. Теперь то же самое надо сделать с числом прослушанных меломаном композиций. Отсортируйте данные группировки count_music_user по убыванию. Сохраните результат в переменной final_count, значение которой выведите на экран.


In [ ]:
# задание 1
print('ЗАДАНИЕ 1\n')
music_user = yandex_music[(yandex_music['user_id'] == search_id) & (yandex_music['total_play_seconds'] != 0)]
print(music_user)

# задание 2
print('ЗАДАНИЕ 2')
sum_music_user = music_user.groupby('genre_name')['total_play_seconds'].sum()
print(sum_music_user)

# задание 3
print('\nЗАДАНИЕ 3\n')
count_music_user = music_user.groupby('genre_name')['genre_name'].count()
print(count_music_user)


ЗАДАНИЕ 1

        user_id track_name artist_name  genre_name              city  \
274    7D166C63    unknown     unknown        jazz  Saint-Petersburg   
2375   7D166C63    unknown     unknown         pop  Saint-Petersburg   
2561   7D166C63    unknown     unknown     country  Saint-Petersburg   
3096   7D166C63    unknown     unknown    schlager  Saint-Petersburg   
4334   7D166C63    unknown     unknown        jazz  Saint-Petersburg   
5671   7D166C63    unknown     unknown         pop  Saint-Petersburg   
6498   7D166C63    unknown     unknown         pop  Saint-Petersburg   
7187   7D166C63    unknown     unknown         pop  Saint-Petersburg   
8388   7D166C63    unknown     unknown         pop  Saint-Petersburg   
8906   7D166C63    unknown     unknown        jazz  Saint-Petersburg   
9408   7D166C63    unknown     unknown        rock  Saint-Petersburg   
11810  7D166C63    unknown     unknown         pop  Saint-Petersburg   
13593  7D166C63    unknown     unknown        rock  S

In [ ]:
# задание 4
print('\nЗАДАНИЕ 4\n')
final_sum = sum_music_user.sort_values(ascending=False)
print(final_sum)

# задание 5
print('\nЗАДАНИЕ 5\n')
final_count = count_music_user.sort_values(ascending=False)
print(final_count)



ЗАДАНИЕ 4

genre_name
pop           20:34:4408:46:5114:23:3008:25:3314:22:4309:04:...
hip                                                    20:29:56
jazz          20:29:5314:23:2714:16:1014:23:0608:39:3914:09:...
classical                                              20:26:32
folkrock                                               14:29:08
rock                   14:12:0214:23:4308:35:5808:56:3008:35:58
dance                                          14:11:5614:23:32
soundtrack                                             14:09:56
folk                                                   13:52:18
punk                                                   13:52:12
reggae                                         08:46:5509:04:14
schlager               08:42:5508:39:3814:13:5409:04:1908:35:45
country                                                08:39:24
ghazal                                                 08:32:18
rnb                                                    08:32:10
Name: total_play_

In [ ]:

zadacha = yandex_music[(yandex_music['genre_name'] == 'folk') & (yandex_music['city'] == 'Moscow')&(yandex_music['day'] == 'Monday')]
print(zadacha['user_id'].count())



291
